# Engenharia do Conhecimento 2022/2023

## TP04: Linear models for Regression and Classification

*A Machine Learning Tutorial by Andre Falcao (DI/FCUL 2021-2022)*

*Revised by Catia Pesquita (2022-2023) and by Luís Correia (2023-2024)*

### Summary

1. The essence of Linear Regression
2. Linear Regression in Scikit-Learn
3. Regularized Models: Ridge Regression and Lasso
4. Logistic Regression for Classification 


## 1. The essence of Linear Regression

### 1.1. Computing Linear regression manually


We know that the parameters of a linear model can be computed according to the equation

$\beta = (X^T.X)^{-1}.X^T.Y$ &emsp; &emsp; &emsp; (1)

Let's check it out for the [diabetes dataset form the toy examples of scikit learn](https://scikit-learn.org/stable/datasets/toy_dataset.html) that has 10 independent variables


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
X_diabetes, y_diabetes=load_diabetes(return_X_y=True)

#show data set, with target values in first column
pd.DataFrame(X_diabetes, y_diabetes)

To use this procedure we first need to add one `[1]` column to the whole dataset that will emcompass the bias of the model (intercept)

In [ ]:
X1 = np.insert(X_diabetes, 0, 1, axis=1)
X1.shape
pd.DataFrame(X1)

Now we will divide the full dataset into training and testing

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X1, y_diabetes, test_size=0.2, random_state=22)


the following procedure just computes the Beta parameters by sequentially solving equation (1) above

In [ ]:
Xtt=np.transpose(X_train) #Xtt - the transposed X matrix (XT)
print(Xtt.shape) #<-check the matrix dimensions
gram=np.dot(Xtt, X_train) #the dot product between XT and X (XT.X)
print(gram.shape) #<-check the matrix dimensions
gram_inv=np.linalg.inv(gram) #now we invert: (XT.X)^-1
print(gram_inv.shape) #<-check the matrix dimensions
X_part = np.dot(gram_inv,Xtt) #the dot product (XT.X)^-1 .XT
print(X_part.shape) #<-check the matrix dimensions 
Beta_est=np.dot(X_part, y_train) #and the final dot product with the output vector XT (XT.X)^-1 .XT .Y

print(Beta_est) #the values of all betas for 0...N

#print("The bias is: %9.3f" % Beta_est[0]) #this is the intercept, beta_zero, or alpha 
print("The parameters are (notice that the first, B0, is the intercept): ") #the coefficients 
for i, beta in enumerate(Beta_est[0:]):
    print("\t B%2d -> %9.3f"% (i, beta))


### 1.2 Predictions with the linear regression model

Making predictions is trivial. It is just a matrix multiplication of the parameters with the dataset for which we want to make predictions

In [ ]:
import matplotlib.pyplot as plt
#computes predicted values by taking the x values and muliplying be the corresponding beta coefficients
my_preds=np.dot(X_test,Beta_est) 

#histogram of predicted values
plt.hist(my_preds, bins=20, alpha=.8)
plt.xlabel("predicted values")
plt.ylabel("count")
plt.grid()
plt.show()

Let's compute some usual regression statistics

In [ ]:
from sklearn.metrics import r2_score, root_mean_squared_error, max_error, mean_absolute_error
from scipy.stats import pearsonr

def printRegStatistics(truth, preds):
    print("The R2 is: ", r2_score(truth, preds))
    print("The rmse is: ", root_mean_squared_error(truth, preds))
#    print("The rmse is: ", mean_squared_error(truth, preds, squared=False)) # this call is deprecated
    corr, pval = pearsonr(truth, preds)
    print("The Correlation Score is is: %6.4f (p-value=%e)\n"%(corr,pval))
    print("The Maximum Error is is: ", max_error(truth, preds))
    print("The Mean Absolute Error is: ", mean_absolute_error(truth, preds))

printRegStatistics(y_test, my_preds)

And compare graphically the results to the actual y values (y_test) 

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(y_test, my_preds)
plt.grid()
#this is the 45degrees angle. The closer the predictions approach this, the better the model
plt.plot([0, 350], [0, 350], c="r")
plt.xlabel("real values")
plt.ylabel("predictions")
plt.show()

## 2. Linear regression with Python libraries

### 2.1 Using scikit-learn

The only difference to the above procedure that we have used for fitting our model is that here we will use the original X matrix and not X1 (which has an extra **ONE** column), so we will need to do a new train-test split this time with the original data

In [ ]:
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(X_diabetes, y_diabetes, test_size=0.2, random_state=22)

reg = LinearRegression().fit(X_train, y_train)

print("The bias is: ",  reg.intercept_)
print("The other parameters are: ")
for i, beta in enumerate(reg.coef_):
    print("\t B%2d -> %9.3f"% (i+1, beta))
    
print("The score is: ", reg.score(X_train, y_train))

Compare the results of the parameter values with the ones computed above, at the end of 1.1. They should be exactly the same! A linear regression is a stable model - always gives the same result.

## Exercise 1
A couple of questions

In [ ]:
print("The score displayed above is the value of metric: ", end=" ")
print("**<your answer here>**")
print("")
print("It has been computed over the: ", end=" ")
print("**<your answer here>**", "set")

### ... finishing the comparisons

Similarly let's examine the predictions graphically (should also be identical to previous)

In [ ]:
preds=reg.predict(X_test)
plt.figure(figsize=(7,7))
plt.scatter(y_test, preds)
plt.grid()
plt.plot([0, 350], [0, 350], c="r")
plt.xlabel("real values")
plt.ylabel("predictions")
plt.show()

## 3. Regularised linear models

Regularised models impose boundaries on the fitted parameters, so that it is possible to have sensitivity on which variables appear most relevant for explaining the dependent `y`. The problem is that these boundaries add constraints and simple optimization of the MSE is no longer possible, requiring more complex fitting algorithms

There are essentially 3 regularised linear models
* Ridge Regression
* Lasso (Least Absolute Shrinkage and Selection Operator)
* Elastic nets

For this class we will only discuss Ridge and Lasso, as Elastic nets are a weighted compromise betwen these two approaches. 

### 3.1 Ridge Regression

Ridge regression applies L2 regularization, that is, it minimizes not only the MSEs (mean squared errors), but also the squared values of the parameters estimator, thus:

$cost(\beta) = MSE(\beta) + \frac{\alpha}{2}\sum^{n}_{i=1}{\beta_i^2}$

The value of $\alpha$ defines how much to penalize the parameter values. This penalty will emphasize the importance of each variable in its contribution for explaining `y` Higher values of $\alpha$, will cause smaller $\beta$

Selecting a value of  $\alpha=10$

In [ ]:
from sklearn.linear_model import Ridge, Lasso
ridge = Ridge(alpha=10, max_iter=9999999).fit(X_train, y_train)

print("The bias is: ",  ridge.intercept_)
print("The other parameters are: ")
for i, beta in enumerate(ridge.coef_):
    print("\t B%2d -> %9.3f"% (i+1, beta))
    

We can of course compute the general regression statistics for the regularized model

In [ ]:
preds10=ridge.predict(X_test)
printRegStatistics(y_test, preds10)

Changing the $\alpha$ value will have a dramatic impact on the parameters estimated as well as on the regression results. for instance for  $\alpha=0.1$ we will have

In [ ]:
ridge = Ridge(alpha=0.06, max_iter=9999999).fit(X_train, y_train)

print("The bias is: ",  ridge.intercept_)
print("The other parameters are: ")
for i, beta in enumerate(ridge.coef_):
    print("\t B%02d -> %9.3f"% (i+1, beta))
preds0_1=ridge.predict(X_test)
printRegStatistics(y_test, preds0_1)

For very low $\alpha$ values the model is equivalent to the unconstrained model

In [ ]:
ridge = Ridge(alpha=0.0001, max_iter=9999999).fit(X_train, y_train)

print("The bias is: ",  ridge.intercept_)
print("The other parameters are: ")
for i, beta in enumerate(ridge.coef_):
    print("\t B%2d -> %9.3f"% (i+1, beta))
preds0_0001=ridge.predict(X_test)
printRegStatistics(y_test, preds0_0001)

Now is there any value of $\alpha$ that actually is better for the current train/test partition? Let's check the R2 metric for different values and plot the results, by using grid search (test all the values of $\alpha$ in an array).

In [ ]:
from sklearn.model_selection import GridSearchCV

ridge = Ridge(random_state=0, max_iter=10000)
#lasso = Lasso(random_state=0, max_iter=10000)
alphas=2**np.arange(-0,-10,-.05)
tuned_parameters = [{"alpha": alphas}]
n_folds = 5

#cf = GridSearchCV(ridge, tuned_parameters, cv=n_folds, refit=False)
cf = GridSearchCV(ridge, tuned_parameters, cv=n_folds, return_train_score=True, refit=False)
cf.fit(X_train, y_train)
tr_scores = cf.cv_results_["mean_train_score"]
tr_scores_std = cf.cv_results_["std_train_score"]
te_scores = cf.cv_results_["mean_test_score"]
te_scores_std = cf.cv_results_["std_test_score"]

plt.figure().set_size_inches(8, 6)
plt.semilogx(alphas, tr_scores, label="train scores")
plt.semilogx(alphas, te_scores, label="test scores")

tr_std_error = tr_scores_std / np.sqrt(n_folds)
te_std_error = te_scores_std / np.sqrt(n_folds)

plt.semilogx(alphas, tr_scores + tr_std_error, "g--")
plt.semilogx(alphas, tr_scores - tr_std_error, "g--")
plt.semilogx(alphas, te_scores + te_std_error, "b--")
plt.semilogx(alphas, te_scores - te_std_error, "b--")

# alpha=0.2 controls the translucency of the fill color
plt.fill_between(alphas, tr_scores + tr_std_error, tr_scores - tr_std_error, alpha=0.2)
plt.fill_between(alphas, te_scores + te_std_error, te_scores - te_std_error, alpha=0.2)

plt.ylabel("CV score (R2) +/- std error")
plt.xlabel("alpha")
plt.axhline(np.max(tr_scores), linestyle="--", color=".3")
plt.axhline(np.max(te_scores), linestyle="--", color=".5")
best_alpha=next(iter(cf.best_params_.values()))
plt.axvline(x = best_alpha, color = 'b', label = 'best Alpha')
plt.legend();
print("Best $alpha$ value: ", best_alpha)

In [ ]:
#this is a simple cross-validation
# it shows that results are not the same as with cross validation

r2_train = []
r2_test = []
#rmse_train = []
#rmse_test = []
#generate 160 values of alpha in [2^(-2)...2^(-10)]
alphas=2**np.arange(-2,-10,-.05)
for alpha in alphas:
    ridge = Ridge(alpha=alpha, max_iter=9999999).fit(X_train, y_train)
    preds_tr=ridge.predict(X_train)
    preds_te=ridge.predict(X_test)
    r2_train.append(r2_score(y_train, preds_tr))
    r2_test.append(r2_score(y_test, preds_te))
#    rmse_train.append(root_mean_squared_error(y_train, preds_tr))
#    rmse_test.append(root_mean_squared_error(y_test, preds_te))
#    rmse_train.append(mean_squared_error(y_train, preds_tr, squared=False))  # deprecated function call
#    rmse_test.append(mean_squared_error(y_test, preds_te, squared=False))  # deprecated function call
    
plt.semilogx(alphas, r2_train, label="Train")    
plt.semilogx(alphas, r2_test, label="Test") 
#plt.plot(alphas, rmse_train, label="rmse Train")    
#plt.plot(alphas, rmse_test, label="rmse Test") 
plt.xlabel("alpha")
plt.ylabel("R2")
plt.grid()
plt.legend()
plt.show()

it is possible to inspect the values of the different parameters as $\alpha$ changes

In [ ]:
#compute coefficients for many alpha values
coefs=[]

#alternative: 300 different values of alpha in [2^5...2^(-10)]
alphas=2**np.arange(5,-10,-.05) 
for alpha in alphas:
    ridge= Ridge(alpha=alpha, max_iter=100000).fit(X_train, y_train)
    coefs.append(ridge.coef_)
coefs=np.array(coefs)

#plot results
N,M=X_train.shape #get number of rows(N) which correspond to alphas and columns(M) which correspond to the variables)
plt.figure(figsize=(10,7))
for i in range(M):
    plt.plot(alphas, coefs[:,i], label="Var %d" % (i+1)) #the coefficents of variable i for each alpha value
    
plt.xscale("log")
plt.legend()
plt.xlabel("alpha")
plt.ylabel("coefficients")
plt.grid()
plt.show()

We can use scikit-learn own RidgeCV to find the best $\alpha$ value using cross-validation

In [ ]:
#lets find the best alpha using cross-validation on the training data  
from sklearn.linear_model import RidgeCV
ridge_cv = RidgeCV(alphas = alphas, cv=5).fit(X_train, y_train)
print("The best alpha is: ",ridge_cv.alpha_)

#plot results
N,M=X_train.shape #get number of rows(N) which correspond to alphas and columns(M) which correspond to the variables)
plt.figure(figsize=(10,7))
for i in range(M):
    plt.plot(alphas, coefs[:,i], label="Var %d" % (i+1)) #the coefficents of variable i for each alpha value
plt.axvline(x = ridge_cv.alpha_, color = 'b', label = 'best Alpha')
plt.xscale("log")
plt.legend()
plt.xlabel("alpha")
plt.ylabel("coefficients")
plt.grid()
plt.show()

In [ ]:
#Now let's fit the linear regression using the best alpha and evaluate on the test set
ridge = Ridge(alpha=ridge_cv.alpha_, max_iter=100000).fit(X_train, y_train)
preds_best=ridge.predict(X_test)
printRegStatistics(y_test, preds_best)


## Exercise 2:
### Compare the results obtained with alpha=10, alpha=0.0001 and the best alpha. What can you conclude?

In [ ]:
# Notice: all these values have been computed before...
#insert your code here


In [ ]:
print("**<your answer here>**")

### 3.2 The Lasso

The Lasso (Least Absolute Shrinkage and Selection Operator) is similar to Ridge regression but it uses L1 regularization, that is, it minimizes not only the MSEs, but including the modulus values of the parameters estimators, which will have a very important effect on the actual parameters:

$cost(\beta) = MSE(\theta) + \alpha\sum^{n}_{i=1}{|\beta_i|}$

This will make that only really necessary variables will enter the model. Similar to Ridge regression the  $\alpha$ parameter defines how much to penalize the actual parameters. This penalization will emphasize the importance of each variable in its contribution for explaining `y`. 
Higher values of $\alpha$, will cause smaller $\beta$. However different from Ridge regression, due to the nature of the modulus function, for high alpha values only a few variables will enter the model. This is a very important criterion for actually identifying the most important variables in a model.

Selecting a value of  $\alpha=2$, for instance, will only have two variables in the model (\=0), suggesting they are the most important ones for explaining `y`.

In [ ]:
L = Lasso(alpha=2, max_iter=9999999).fit(X_train, y_train)
print("The bias is: ",  L.intercept_)
print("The other parameters are: ")
for i, beta in enumerate(L.coef_):
    print("\t B%2d -> %9.3f"% (i+1, beta))
preds=L.predict(X_test)
printRegStatistics(y_test, preds)

Making  $\alpha=0.5$ will impact the model, showing now 4 active variables and an apparent increase in the model statistics

In [ ]:
L = Lasso(alpha=.5, max_iter=9999999).fit(X_train, y_train)

print("The bias is: ",  L.intercept_)
print("The other parameters are: ")
for i, beta in enumerate(L.coef_):
    print("\t B%02d -> %9.3f"% (i+1, beta))
preds=L.predict(X_test)
printRegStatistics(y_test, preds)

## Exercise 3:
Repeat the procedure above described for Ridge regression, but now adapted for the Lasso.

### 3.1 Use cross-validation to decide on the best alpha value 
(suggestion: look for the corresponding function in scikit-learn).

In [ ]:
#lets find the best alpha using cross-validation on the training data... 


### 3.2 See how the best alpha value impacts the values of the coefficients. How many are not zero?

In [ ]:
# your code here


### 3.3 Plot R2 vs. alpha values and print the best alpha value

In [ ]:
# your code here


### 3.4 Plot the distribution of alpha values and coefficients.

In [ ]:
# your code here


## 4. Logistic regression

Despite the name, Logistic regression only uses regression methods for fitting, but what is fitted is the probability of belonging to a given class, making this a classification method. This probability function is totaly non-linear.


### 4.1. A very simple explanation of Logistic Regression

We will explain Logistic Regression using articially created data in two different files. One with positive and the other with negative samples.

In [ ]:
lines=np.genfromtxt('pos_smps.txt')
poss=np.array([float(lin) for lin in lines]).reshape(-1, 1) #array of positive examples
Np=poss.shape[0] #number of positive examples
lines=np.genfromtxt('neg_smps.txt')
negs=np.array([float(lin) for lin in lines]).reshape(-1, 1) #array of negative examples
Nn=negs.shape[0]  #number of positive examples

yp=np.ones(Np) #array of labels for positive examples
yn=np.zeros(Nn) #array of labels for negative example

In [ ]:
#plot the examples. for positives: y=1, for negatives: y=0
plt.scatter(poss, yp, label="positives")
plt.scatter(negs, yn, label="negatives")
plt.ylim([-0.2, 1.2])
plt.legend()
plt.xlabel("X values")
plt.ylabel("y (classification) values")
plt.grid()
plt.show()

We want to fit a logistic regression that will assign a probability that each instance is positive. That probability cab be defined with a logistic function, with a characteristic sigmoid shape:

$p=\dfrac{1}{1+e^{-(\beta_0+\beta_1.x)}}$

this probability logistic curve is the probability curve. For each value of x, according to this logistic function we can compute the probability of each instance being positive. For this case, as it can be seen the largest the X, the highest the probability of it being positive. 

For illustration purposes as an initial guess, we are just going to assume two estimates for $\beta_0$ and $\beta_1$: 

* $\beta_0= -10.0$
* $\beta_1= 4.0$



In [ ]:
#let's plot
b0 =-10.0
b1 = 4.0
x=np.arange(0, 8, 0.01) #x values to plot the
p= 1/(1+np.exp(-(b0+b1*x))) #probability logistic

plt.plot(x, p, c="k")
plt.scatter(poss, yp)
plt.scatter(negs, yn)
plt.xlabel("X values")
plt.ylabel("y (classification) values")
plt.grid()
plt.show()

We can solve the above equation relative to the probability to find that this curve is in fact a linear representation of the log odds. This *log odds* is the logarithm of the ratio that some instance is positive to the probability of it being negative:

$\log(\frac{p}{1-p})=\beta_0+\beta_1.x$

Now the log odds does not support neither the value of `p=1` neither `p=0`, each case implying $+\infty$  and $-\infty$ respectively. However we can project each point in the new line

This can be represented graphically

In [ ]:
pp_lo = b0 + b1*poss  #log odds for positives
pn_lo = b0 + b1*negs  #log odds for negatives

plt.scatter(poss, pp_lo, s = 20)
plt.scatter(negs, pn_lo, s = 20)

plt.axline((0, b0), slope=b1, c="k") #the linear regression

plt.ylim([-8,8])
plt.title("Log Odds plot")
plt.xlabel("log odds of X values")
plt.ylabel("log odds of y (classification) values")
plt.grid()


We could of course project our samples into the logistic curve. The points above the 0.5 line are the points that should be considered that have a higher likelyhood of being positive, and for those below, have a higher likelyhood of being negative

In [ ]:
pp= 1/(1+np.exp(-(b0+b1*poss)))
pn= 1/(1+np.exp(-(b0+b1*negs)))
plt.plot(x, p, c="k")
plt.scatter(poss, pp, s = 20)
plt.scatter(negs, pn, s = 20)
plt.axhline(.5,  linestyle="--", color="r")
plt.xlabel("X values")
plt.ylabel("y (classification) values")
plt.grid()
plt.show()


The predictions are then the sigmoid function values resulting from the log odds which was fitted as a straight line.


In [ ]:
#let's first put all elements in a common X, y structure for later using with scikit-learn
X=np.vstack((poss, negs))
y=np.hstack((yp, yn))

# let's reserve a test set aside, name with l for logistic
Xl_train, Xl_test, yl_train, yl_test = train_test_split(X, y, test_size=0.2, random_state=22)

#now we can make predictions with test set
preds=(1/(1+np.exp(-(b0+b1*Xl_test)))>0.5)
preds=preds.flatten().astype('int32')
preds

How good is this model? We need to compute the typical statistics

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, matthews_corrcoef, confusion_matrix, accuracy_score
import pandas as pd
def printClassResults(truth, preds):
    print("The Accuracy is: %7.4f" % accuracy_score(truth, preds))
    print("The Precision is: %7.4f" % precision_score(truth, preds))
    print("The Recall is: %7.4f" % recall_score(truth, preds))
    print("The F1 score is: %7.4f" % f1_score(truth, preds))
    print("The Matthews correlation coefficient is: %7.4f" % matthews_corrcoef(truth, preds))
    print()
    print("This is the Confusion Matrix")
    print(pd.DataFrame(confusion_matrix(truth, preds)))

printClassResults(yl_test, preds)

## Exercise 4

What if we change the parameters?

Use $\beta_0$ = `-6.0`, and  $\beta_1$ = `2.0`
(to make new predictions is trivial, we just use the new parameters in the formula to get new predictions and print the results)

In [ ]:
#your code here...


### 4.2. Logistic Regression on sickit-learn


Differently from Linear Regression, the process for finding the best possible solution is heuristic, using typically gradient descent approaches. We can use scikit learn for finding the best possible parameters for the above problem

In [ ]:
from sklearn.linear_model import LogisticRegression
mdl = LogisticRegression()
mdl.fit(Xl_train, yl_train)

b0=mdl.intercept_[0]
b1=mdl.coef_[0][0]
print("Intercept (b0): %7.4f"% b0) 
print("Slope     (b1): %7.4f"% b1) 


We can furthermore see the curve and how it compares to the original model with training set used to fit the new curve

In [ ]:
#pp= 1/(1+np.exp(-(b0+b1*poss)))
#pn= 1/(1+np.exp(-(b0+b1*negs)))

#this is the scikit fitted curve
p_sk= 1/(1+np.exp(-(b0+b1*x)))

#below is the original curve
p_or= 1/(1+np.exp(-(-10.0+4.0*x)))

plt.plot(x, p_sk, c="k", label="scikit")
plt.plot(x, p_or, c="b", label="original")
plt.scatter(Xl_train, yl_train, s = 20)
#plt.scatter(poss, yp, s = 20)
#plt.scatter(negs, yn, s = 20)
plt.axhline(.5,  linestyle="--", color="r")
plt.grid()
plt.legend()
plt.show()


Finally we can compare the metrics computing them for the new model, using the test set now

In [ ]:
sk_preds=mdl.predict(Xl_test)
printClassResults(yl_test, sk_preds)

### Discussion

Compare the results obtained and discuss them
    * What to you think is happening in scikit-learn
    * how relevant is the aspect of the curve to the overal statistics. Focus in particular on precision and recall

### 4.2 Scikit-learn example on multidimensional data

Scikit is able to fit logistic regression parameters to multidimensional data and with multiclass objectives. However some care must be taken when there is more than one independent variable. Due to the learning process, it is fundamental that the data is apropriately scaled

Furthermore, now we will use tradidional cross validation to evaluate our models

#### 4.2.1 Scaling data

Due to the process of model fitting it is required that the X matrix has been scaled to 0 mean and variance of 1. Scikit learn has a tool that is able to accomplish this (`StandardScaler`) but we must be careful to maintain a separation between training and testing or the Scaling process will be a source of error.

The correct procedure for scaling data is as follows
1. Separate full data set into training and testing
2. Use the training set to fit the scaler
3. Apply the scaler to the training set (transform)
4. Apply the scaler to the testing set (transform)

We are going to use the Breast Cancer data set from the scikit learn toy data sets

In [ ]:
from sklearn.datasets import load_breast_cancer
X_bc,y_bc = load_breast_cancer(return_X_y=True)
Xl_train, Xl_test, yl_train, yl_test = train_test_split(X_bc, y_bc, test_size=0.2, random_state=22)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(Xl_train) #fit scaler on training set
Xl_train = scaler.transform(Xl_train)  #apply scaler on training set
Xl_test = scaler.transform(Xl_test) #apply scaler on test set

#### 4.2.2. Fitting the model

Now let's fit the model to the training data properly scaled

In [ ]:
mdl= LogisticRegression(random_state=0).fit(Xl_train, yl_train)

We can print the coefficients of the model. Please note that we would require several sets of parameters if the problem is multiclass. For 3 classes we would require 3 parameter sets. This will force us to use a slightly different way to have access to each of the actual model coefficients

Also note that as all variables are centered in zero and with similar variance it is possible to identify the variables that may have strongest impact on the model as these are the ones with higher absolute value on the coeficients

In [ ]:
print("The bias is: ",  mdl.intercept_[0])
print("The other parameters are: ")
for i, beta in enumerate(mdl.coef_[0]):
    print("\t B%02d -> %9.3f"% (i+1, beta))


We may look at the five most influent (those with higher absolute value):

In [ ]:
coefs=[(beta,i) for i, beta in enumerate(mdl.coef_[0])]
coefss=sorted(coefs, key=lambda row: np.abs(row[0]))
coefss.reverse()
for beta, i in coefss[:5]:
    print("\t B%02d -> %9.3f"% (i+1, beta))

And we can make predictions and evaluation with the test set

In [ ]:
preds = mdl.predict(Xl_test)

printClassResults(yl_test, preds)